## Clone repo and import dependencies

In [ ]:
!pip install pytorch==1.10.1 tensorboard==2.7.0 pandas
import os 
import torch
from torch.autograd.grad_mode import F
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

%cd /tmp
%rm -rf /tmp/FlowPredict
!git clone https://github.com/davidharrod/FlowPredict.git
%cd /tmp/FlowPredict
import modules
import data_utils

## Prepare dataset

In [ ]:
file_dir = "./"
batch_size = 50
shuffle=True
data = data_utils.read_files(file_dir)
dataset = modules.Rk4Dataset(data)
data_loader = DataLoader(dataset,batch_size=batch_size,shuffle=shuffle,drop_last=True)


## Set up training

In [ ]:
learning_rate = 0.01
# Set up device.
device = "cuda" if torch.cuda.is_available() else "cpu"
loss_fn = nn.MSELoss()
model = modules.TestNN().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

## Start training

In [ ]:
epoch = 100
check_step = 10

size = len(data_loader.dataset)
model.train()
tensorboard_dir = os.getcwd()
writer = SummaryWriter(log_dir=tensorboard_dir)
for epoch_count in range(epoch):
    print(f"=====Epoch {epoch_count + 1} Start training=====")
    for batch, (input_tensor, Wt0) in enumerate(data_loader):
        input_tensor, Wt0 = input_tensor.to(device), Wt0.to(device)
        # Compute prediction error.
        pred = model(input_tensor)
        model_loss = loss_fn(pred,Wt0)
        # Backpropagation.
        optimizer.zero_grad()
        model_loss.backward()
        optimizer.step()
        if batch % check_step == 0:
            writer.add_scalar("loss", model_loss,batch + epoch_count * size / data_loader.batch_size)
            print(f"Current Loss: {model_loss}")
    print(f"=====Epoch {epoch_count + 1} done=====\n")
writer.close()

## Check out tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./